In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm
import yfinance as yf
import mibian
#get stocks to look at
file_path = r'C:\Users\allis\Downloads\portfolio10.csv'
tickers_df = pd.read_csv(file_path)
print(tickers_df)

  ticker
0   EXPE
1     GM
2    USB
3    ACN
4    FIS
5    AXP
6   EBAY
7    AMT
8    MAR
9     HD


In [9]:
# Black-Scholes function for put options price calculation
def black_scholes_put_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Fetch the 10-year Treasury yield for the risk-free rate
treasury_ticker = yf.Ticker("^TNX")
risk_free_rate = treasury_ticker.history(period="1d")['Close'].iloc[0] / 100  # Convert to decimal

# Prepare DataFrames to store put option prices and Greeks
all_puts_data = pd.DataFrame()
all_puts_greeks = pd.DataFrame()

# Loop through each ticker in the portfolio
for ticker_symbol in tickers_df['ticker']:
    try:
        # Fetch stock data and options chain from yfinance
        ticker = yf.Ticker(ticker_symbol)
        current_price = ticker.history(period="1d")['Close'].iloc[0]

        # Retrieve options chain for each expiration date
        expiry_dates = ticker.options
        for expiry_date in expiry_dates:
            try:
                # Get put options for the specified expiration date
                options_chain = ticker.option_chain(expiry_date)
                puts = options_chain.puts

                # Calculate time to expiration in days and years
                expiration_date = datetime.strptime(expiry_date, "%Y-%m-%d")
                days_until_expiration = (expiration_date - datetime.now()).days
                time_to_expiration = days_until_expiration / 365

                # Loop through each put option to calculate the Black-Scholes price and Greeks
                for index, row in puts.iterrows():
                    strike_price = row['strike']
                    implied_volatility = row['impliedVolatility']

                    # Skip options with missing or zero implied volatility
                    if pd.isna(implied_volatility) or implied_volatility == 0:
                        continue

                    # Calculate Black-Scholes price
                    put_price = black_scholes_put_price(
                        S=current_price,
                        K=strike_price,
                        T=time_to_expiration,
                        r=risk_free_rate,
                        sigma=implied_volatility
                    )

                    # Calculate Greeks using Mibian
                    option = mibian.BS(
                        [current_price, strike_price, risk_free_rate, days_until_expiration],
                        volatility=implied_volatility * 100 
                    )
                    delta = option.putDelta
                    gamma = option.gamma
                    theta = option.putTheta
                    vega = option.vega

                    # Store calculated price and Greeks
                    option_data = pd.DataFrame([{
                        "ticker": ticker_symbol,
                        "expiration_date": expiry_date,
                        "strike": strike_price,
                        "implied_volatility": implied_volatility,
                        "put_price": put_price,
                        "current_price": current_price,
                        "Delta": delta,
                        "Gamma": gamma,
                        "Theta": theta,
                        "Vega": vega
                    }])

                    # Append both price and Greeks to the main DataFrame
                    all_puts_data = pd.concat([all_puts_data, option_data], ignore_index=True)

            except Exception as e:
                print(f"Error processing options for {ticker_symbol} on expiry {expiry_date}: {e}")
                continue

    except Exception as e:
        print(f"Error processing {ticker_symbol}: {e}")

# Display results
print("All Put Options Data with Greeks:")
print(all_puts_data)

All Put Options Data with Greeks:
     ticker expiration_date  strike  implied_volatility  put_price  \
0      EXPE      2024-11-29   110.0            1.986328   0.059084   
1      EXPE      2024-11-29   125.0            1.529299   0.056880   
2      EXPE      2024-11-29   130.0            1.519534   0.114076   
3      EXPE      2024-11-29   135.0            1.464846   0.176846   
4      EXPE      2024-11-29   139.0            1.171879   0.065670   
...     ...             ...     ...                 ...        ...   
3030     HD      2027-01-15   410.0            0.199204  26.187534   
3031     HD      2027-01-15   420.0            0.200592  30.252583   
3032     HD      2027-01-15   450.0            0.182259  38.918707   
3033     HD      2027-01-15   460.0            0.180924  43.432965   
3034     HD      2027-01-15   540.0            0.146844  84.563133   

      current_price     Delta     Gamma     Theta      Vega  
0        184.919998 -0.004633  0.000351 -0.064937  0.002615  
1

In [15]:
for ticker_symbol in all_puts_data['ticker'].unique():
    ticker_puts = all_puts_data[all_puts_data['ticker'] == ticker_symbol]

    # Find the best long put (strike >= current price)
    long_put = ticker_puts[ticker_puts['strike'] >= ticker_puts['current_price']].sort_values('strike').iloc[0]

    # Find the best short put (strike < current price)
    short_put = ticker_puts[ticker_puts['strike'] < ticker_puts['current_price']].sort_values('strike', ascending=False).iloc[0]

    # Calculate net debit
    net_debit = long_put['put_price'] - short_put['put_price']

    # Calculate max profit, max loss, and breakeven price
    max_profit = short_put['strike'] - long_put['strike'] - net_debit
    max_loss = net_debit
    breakeven_price = long_put['strike'] - net_debit

    # Clean and format the output
    print(f"\nTicker: {ticker_symbol}")
    print(f"Long Put: Expiration: {long_put['expiration_date']}, Strike: {long_put['strike']}, "
          f"Implied Volatility: {long_put['implied_volatility']:.2%}, Price: ${long_put['put_price']:.2f}")
    print(f"Short Put: Expiration: {short_put['expiration_date']}, Strike: {short_put['strike']}, "
          f"Implied Volatility: {short_put['implied_volatility']:.2%}, Price: ${short_put['put_price']:.2f}")
    print(f"Net Debit: ${net_debit:.2f}")
    print(f"Max Profit: ${max_profit:.2f}")
    print(f"Max Loss: ${max_loss:.2f}")
    print(f"Breakeven Price: ${breakeven_price:.2f}")


Ticker: EXPE
Long Put: Expiration: 2024-11-29, Strike: 185.0, Implied Volatility: 24.00%, Price: $1.85
Short Put: Expiration: 2024-11-29, Strike: 182.5, Implied Volatility: 24.49%, Price: $0.88
Net Debit: $0.96
Max Profit: $-3.46
Max Loss: $0.96
Breakeven Price: $184.04

Ticker: GM
Long Put: Expiration: 2024-11-29, Strike: 59.0, Implied Volatility: 30.81%, Price: $1.00
Short Put: Expiration: 2024-11-29, Strike: 58.0, Implied Volatility: 26.37%, Price: $0.40
Net Debit: $0.60
Max Profit: $-1.60
Max Loss: $0.60
Breakeven Price: $58.40

Ticker: USB
Long Put: Expiration: 2025-04-17, Strike: 52.5, Implied Volatility: 32.61%, Price: $3.80
Short Put: Expiration: 2024-12-27, Strike: 52.0, Implied Volatility: 22.36%, Price: $1.06
Net Debit: $2.74
Max Profit: $-3.24
Max Loss: $2.74
Breakeven Price: $49.76

Ticker: ACN
Long Put: Expiration: 2024-11-29, Strike: 360.0, Implied Volatility: 20.08%, Price: $3.63
Short Put: Expiration: 2024-12-20, Strike: 357.5, Implied Volatility: 30.82%, Price: $10.4